In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from utils import generate_constrained_random

In [3]:
data = pd.read_csv('../../../Dataset/Spambase/spambase.data.txt', header=None)

In [4]:
data.head(2)

,0,1,2,3,4,5,6,7,8,9,...,48,49,50,51,52,53,54,55,56,57
0,0.00,0.64,0.64,0.0,0.32,0.00,0.00,0.00,0.0,0.00,...,0.0,0.000,0.0,0.778,0.00,0.000,3.756,61,278,1
1,0.21,0.28,0.50,0.0,0.14,0.28,0.21,0.07,0.0,0.94,...,0.0,0.132,0.0,0.372,0.18,0.048,5.114,101,1028,1


In [5]:
y = data.pop(57).values
X = data.values
print X.shape, y.shape

(4601, 57) (4601,)


In [6]:
num_features = X.shape[1]

In [7]:
X_random = generate_constrained_random(X, feature_names=data.columns, data_size=1000)
print X_random.shape

(1000, 57)


In [8]:
import tensorflow as tf

In [9]:
tf.reset_default_graph()

In [10]:
with tf.variable_scope("Placeholders") as scope:
    input_email = tf.placeholder(dtype=tf.float32, shape=[None, num_features], name="input_email")
    label = tf.placeholder(dtype=tf.float32, shape=[None, 2], name="labels")

In [11]:
num_hidden = 128
with tf.variable_scope("hidden_layer") as scope:
    w = tf.get_variable(name="weights", shape=[num_features, num_hidden], dtype=tf.float32, initializer=tf.random_normal_initializer())
    b = tf.get_variable(name="biases", shape=[num_hidden], initializer=tf.random_normal_initializer(), dtype=tf.float32)
    act = tf.matmul(input_email, w) + b
    out = tf.nn.relu(act)

In [12]:
with tf.variable_scope("output_layer") as scope:
    w = tf.get_variable(name="weights", shape=[num_hidden, 2], dtype=tf.float32, initializer=tf.random_normal_initializer())
    b = tf.get_variable(name="biases", shape=[2], dtype=tf.float32, initializer=tf.random_normal_initializer())
    softmax = tf.nn.softmax(tf.matmul(out, w) + b)

In [13]:
with tf.variable_scope("loss") as scope:
    loss = tf.reduce_mean(tf.reduce_sum(-label*tf.log(softmax + 1e-10), reduction_indices=[1]))

In [14]:
with tf.variable_scope("optimizer") as scope:
    optimizer = tf.train.AdamOptimizer(learning_rate=1e-4).minimize(loss)

In [15]:
with tf.variable_scope("accuracy") as scope:
    acc = tf.reduce_mean(tf.cast(tf.equal(tf.arg_max(softmax, 1), tf.arg_max(label, 1)), dtype=tf.float32)) * 100

In [16]:
N_EPOCHS = 2000
SKIP_STEP = 200
with tf.Session() as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
    
    for i in range(1, N_EPOCHS+1):
        
        _, l, accuracy = sess.run([optimizer, loss, acc], feed_dict={input_email:X, label:pd.get_dummies(y).values})
        
        
        if i % SKIP_STEP == 0:
            
            print 'Iteration: {}\n Loss: {}\t Accuracy: {}'.format(i, l, accuracy)
            
    
    y_random = sess.run([softmax], feed_dict={input_email:X_random.values})

Iteration: 200
 Loss: 8.78439426422	 Accuracy: 58.5524864197
Iteration: 400
 Loss: 8.25270175934	 Accuracy: 61.117149353
Iteration: 600
 Loss: 7.9855837822	 Accuracy: 62.8776359558
Iteration: 800
 Loss: 7.23490905762	 Accuracy: 65.3118896484
Iteration: 1000
 Loss: 3.78324079514	 Accuracy: 76.8528594971
Iteration: 1200
 Loss: 2.71056008339	 Accuracy: 82.6559448242
Iteration: 1400
 Loss: 2.13897633553	 Accuracy: 85.785697937
Iteration: 1600
 Loss: 1.67868840694	 Accuracy: 88.046081543
Iteration: 1800
 Loss: 1.43331038952	 Accuracy: 89.2414703369
Iteration: 2000
 Loss: 1.09237408638	 Accuracy: 91.0019607544


In [17]:
y_random = y_random[0]
print y_random.shape
print type(y_random)

(1000, 2)
<type 'numpy.ndarray'>


In [21]:
tf.reset_default_graph()

with tf.variable_scope("Placeholders") as scope:
    input_email = tf.placeholder(dtype=tf.float32, shape=[None, num_features], name="input_email")
    label = tf.placeholder(dtype=tf.float32, shape=[None, 2], name="labels")
    
num_hidden = 512
with tf.variable_scope("hidden_layer") as scope:
    w = tf.get_variable(name="weights", shape=[num_features, num_hidden], dtype=tf.float32, initializer=tf.random_normal_initializer())
    b = tf.get_variable(name="biases", shape=[num_hidden], initializer=tf.random_normal_initializer(), dtype=tf.float32)
    act = tf.matmul(input_email, w) + b
    out = tf.nn.relu(act)
    
with tf.variable_scope("output_layer") as scope:
    w = tf.get_variable(name="weights", shape=[num_hidden, 2], dtype=tf.float32, initializer=tf.random_normal_initializer())
    b = tf.get_variable(name="biases", shape=[2], dtype=tf.float32, initializer=tf.random_normal_initializer())
    softmax = tf.nn.softmax(tf.matmul(out, w) + b)
    
with tf.variable_scope("loss") as scope:
    loss = tf.reduce_mean(tf.reduce_sum(-label*tf.log(softmax + 1e-10), reduction_indices=[1]))
    
with tf.variable_scope("optimizer") as scope:
    optimizer = tf.train.AdagradOptimizer(learning_rate=1e-4).minimize(loss)
    
with tf.variable_scope("accuracy") as scope:
    acc = tf.reduce_mean(tf.cast(tf.equal(tf.arg_max(softmax, 1), tf.arg_max(label, 1)), dtype=tf.float32)) * 100

In [24]:
N_EPOCHS = 2000
SKIP_STEP = 200
with tf.Session() as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
    
    for i in range(1, N_EPOCHS+1):
        
        _, l, accuracy = sess.run([optimizer, loss, acc], feed_dict={input_email:X_random.values, 
                                                                     label:y_random})
        
        if i % SKIP_STEP == 0:
            
            print 'Iteration: {}\n Loss: {}\t Accuracy: {}'.format(i, l, accuracy)
            
    
    y_pred = sess.run([softmax], feed_dict={input_email:X})
    

Iteration: 200
 Loss: 12.8944892883	 Accuracy: 44.0
Iteration: 400
 Loss: 12.8944892883	 Accuracy: 44.0
Iteration: 600
 Loss: 12.8944892883	 Accuracy: 44.0
Iteration: 800
 Loss: 12.8944892883	 Accuracy: 44.0
Iteration: 1000
 Loss: 12.8944892883	 Accuracy: 44.0
Iteration: 1200
 Loss: 12.8944892883	 Accuracy: 44.0
Iteration: 1400
 Loss: 12.8944892883	 Accuracy: 44.0
Iteration: 1600
 Loss: 12.8944892883	 Accuracy: 44.0
Iteration: 1800
 Loss: 12.8944892883	 Accuracy: 44.0
Iteration: 2000
 Loss: 12.8944892883	 Accuracy: 44.0


In [23]:
y_random = pd.get_dummies(np.argmax(y_random, 1)).values